## Brown Dwarf Microlensing Event Checker:

We are using 3 catalogs for the objects: Kirkpatrick21 (K21), Best20 (B20), and the Ultracool Sheet (UCS). In the event that a particular object is in multiple catalogs, we use the astrometry from the newer source (K21 > B20 > UCS).

#### Imports:

In [1]:
# add path to module so can import mlfinder code
import sys
sys.path = ['/Users/judahluberto/mlfinder'] + sys.path

# ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# basic imports
from astropy.io import ascii
from astropy.table import Table
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u

import math

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import statistics

from PyAstronomy import pyasl

import os

# datalab imports for background stars
import dl
from dl import queryClient as qc
from dl.helpers.utils import convert

# import mlfinder module
from mlfinder.bd import BrownDwarf
from mlfinder.fields import Fields
from mlfinder.events import FindEvents

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


#### Let's grab the objects from files we've downloaded from the papers:

In [2]:
file_path = '/Users/judahluberto/python/'

Starting with K21:

In [3]:
# grab files
high_qual_bd = pd.DataFrame(np.array(ascii.read(file_path + 'datafile5.txt')))
low_qual_bd = pd.DataFrame(np.array(ascii.read(file_path + 'datafile6.txt')))

# changing some of their column names for uniformity
rename_dict_k21 = {'FName':'object_name', 
               'RAdeg':'ra', 
               'DEdeg':'dec',
               't0': 't0',
               'plx':'pi', 
               'pmRA':'mu_alpha', 
               'pmDE':'mu_delta', 
               'e_RAdeg':'pm_ra',
               'e_DEdeg':'pm_dec',
               'e_pmRA':'pm_mu_alpha',
               'e_pmDE':'pm_mu_delta',
               'e_plx': 'pm_pi'}


high_qual_bd = high_qual_bd.rename(columns = rename_dict_k21)
low_qual_bd = low_qual_bd.rename(columns = rename_dict_k21)

# now keep those columns too -- for simplicity!
keep_list_k21 = ['object_name', 'ra', 'dec', 't0', 'pi', 'mu_alpha', 'mu_delta', 'pm_ra', 'pm_dec', 'pm_mu_alpha', 'pm_mu_delta', 'pm_pi']

high_qual_bd = high_qual_bd[keep_list_k21]
low_qual_bd = low_qual_bd[keep_list_k21]

# combine the two for simplicity
k21_obj = high_qual_bd.append(low_qual_bd, ignore_index=True)

# convert error in ra and dec from milliarcseconds to degrees
conversion = (3600 * 1000) # number of milliarcseconds in a degree

k21_obj.pm_ra /= conversion
k21_obj.pm_dec /= conversion

Now for B20:

In [4]:
# file
b20_obj = pd.DataFrame(np.array(ascii.read(file_path + 'ajab84f4t4_mrt.txt')))

# rename
rename_dict_b20 = {'Object':'object_name', 
               'RAdeg':'ra', 
               'DEdeg':'dec', 
               'Epoch': 't0',
               'omega-rel':'pi', 
               'pmRA-rel':'mu_alpha', 
               'pmDE-rel':'mu_delta', 
               'e_pmRA-rel':'pm_mu_alpha',
               'e_pmDE-rel':'pm_mu_delta',
               'e_omega-rel': 'pm_pi'}


b20_obj = b20_obj.rename(columns = rename_dict_b20)

# add ra and dec error columns with 0 bc B20 didnt have positional uncertainty
b20_obj['pm_ra'] = 0
b20_obj['pm_dec'] = 0

# keep columns
keep_list_b20 = ['object_name', 'ra', 'dec', 't0', 'pi', 'mu_alpha', 'mu_delta', 'pm_ra', 'pm_dec', 'pm_mu_alpha', 'pm_mu_delta', 'pm_pi']

b20_obj = b20_obj[keep_list_b20]

# convert error in ra and dec from milliarcseconds to degrees
conversion = (3600 * 1000) # number of milliarcseconds in a degree

b20_obj.pm_ra /= conversion
b20_obj.pm_dec /= conversion

For UCS:

- Note that not filtering any non-exoplanet objects, and are entrusting the authors of UCS in the selection of the objects

In [48]:
# file
ultracool_file = '/Users/judahluberto/Downloads/UltracoolSheet - Main.csv'

ucs_obj = pd.read_csv(ultracool_file)

# rename
rename_dict_ucs = {'name':'object_name', 
               'ra_today_formula':'ra', 
               'dec_today_formula':'dec',
               'plx_formula':'pi', 
               'pmra_formula':'mu_alpha', 
               'pmdec_formula':'mu_delta',
               'plxerr_formula': 'pm_pi',
               'pmraerr_formula': 'pm_mu_alpha',
               'pmdecerr_formula': 'pm_mu_delta',}

ucs_obj = ucs_obj.rename(columns = rename_dict_ucs)

# remove objects that they define as exoplanets or subdwarfs
ucs_obj = ucs_obj[~ucs_obj.exoplanet.str.contains('Y', na=False)]

# remove objects that have no parallax
ucs_obj = ucs_obj[ucs_obj.pi > 0]

# make copy of table for later
ucs_obj_all = ucs_obj.copy()

# keep list
keep_list_ucs = ['object_name', 'ra', 'dec', 'pi', 'mu_alpha', 'mu_delta', 'pm_mu_alpha', 'pm_mu_delta', 'pm_pi']

ucs_obj = ucs_obj[keep_list_ucs]

# add ra and dec err of 0 (objects listed do not have errors)
ucs_obj['pm_ra'] = 0
ucs_obj['pm_dec'] = 0

# convert J2000 to ICRS (which is what the legacy survey, our background star source, uses). are close but might as well
ucs_coord = SkyCoord(ucs_obj.ra, ucs_obj.dec, unit='deg', frame='fk5')
ucs_coord = ucs_coord.transform_to('icrs')

ucs_new_ra = ucs_coord.ra.deg
ucs_new_dec = ucs_coord.dec.deg

ucs_obj['ra'] = ucs_new_ra
ucs_obj['dec'] = ucs_new_dec

# add column of t0 to array with 2020-11-01 as the observe date based on the README and the today formulas in UCS
mjds = np.full((1, len(ucs_obj)), 59154)[0]
ucs_obj['t0'] = mjds

Now to combine the catalogs, add a columns showing the reference astrometry, and a column of the date of the data depending on the astrometry:

In [14]:
# combine
all_obj = k21_obj.append([b20_obj, ucs_obj], ignore_index=True).drop_duplicates(subset='object_name')

# drop objects without parallax and 

all_names = all_obj['object_name']
davy_names = list(k21_obj['object_name'])
best_names = list(b20_obj['object_name'])
ultracool_names = list(ucs_obj['object_name'])

refs = list()
for index, value in enumerate(all_names):
    # add to refs list
    if value in davy_names:
        refs.append('K21')
        
    elif value in best_names:
        refs.append('B20')
        
    elif value in ultracool_names:
        refs.append('UCS')
    
# add column to dataframe
all_obj['Ref'] = refs

In [24]:
all_obj[all_obj.object_name == 'WISE J083337.83+005214.2']

,object_name,ra,dec,t0,pi,mu_alpha,mu_delta,pm_ra,pm_dec,pm_mu_alpha,pm_mu_delta,pm_pi,Ref
107,WISE J083337.83+005214.2,128.409249,0.867492,57937.66,79.7,786.8,-1593.7,7.777778e-07,7.777778e-07,2.1,2.0,3.1,K21


#### Run the mlfinder code to see if there are any events:

- Maximum "event" mass uncertainty to 1000 Mjup
- Start and end date at 2014 - 2032. 2014 to catch any events that might've been seen with Gaia, 2032 to look 10 years in future. Any further and we will have telescopes with far better capabilities

We will need to convert dates to proper form, so let's make a function to do that for us first:
- Of course using the clever name "time_to_good" which reformats our time to a good one :)
- init_format is the intial format of the time. Is MJD for object input, but with the MC, it is decimalyear

In [10]:
month_dict = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

def time_to_good(date, init_format='mjd'):
    # create Time instantiation to turn from MJD to iso
    t = Time(float(date), format=init_format)
    t.format = 'iso'
    t = t.value
    
    # split into month, day, year
    t_split = t.split()[0]
    t_split = t_split.split('-')

    # final reformat
    t = t_split[0] + '-' + str(month_dict[int(t_split[1])]) + '-' + t_split[2]
    
    return t

In [18]:
# initliaze a dataframe to append events to, and mc dict of mass uncertainties if done
all_events = pd.DataFrame()
for index, row in all_obj.iterrows():
    # print to keep us updated
    print('Working on dwarf {}, index {}...'.format(row.object_name, index))

    # turn row into a pandas dataframe for ease
    row = row.to_frame().T

    # get time in good format
    t = time_to_good(row.t0)

    # create brown dwarf class and find a future path
    bd = BrownDwarf(row, observ_date=t)
    bd_path = bd.find_path(start='2014-Jan-01', end='2032-Jan-01', step='7days')

    # find the background stars from the Legacy Survey
    stars = Fields(bd=bd)

    # find events with mass uncertainty lower than 1000 Mjup
    events = FindEvents(bd, stars, 1000)

    # append to dataframe
    all_events = all_events.append(events.event_table)

Working on dwarf WISE J000517.48+373720.5, index 0...
Working on dwarf WISE J001505.87-461517.6, index 1...
Working on dwarf PSO J007.9194+33.5961, index 2...
Working on dwarf WISE J003110.04+574936.3, index 3...
Working on dwarf WISE J003231.09-494651.4, index 4...
Working on dwarf 2MASS J00345157+0523050, index 5...
Working on dwarf WISE J003829.05+275852.1, index 6...
Working on dwarf CWISE J004143.77-401929.9, index 7...
Working on dwarf WISE J004542.56+361139.1, index 8...
Working on dwarf WISE J004945.61+215120.0, index 9...
Working on dwarf WISEA J005811.69-565332.1, index 10...
Working on dwarf CWISEP J010527.69-783419.3, index 11...
Working on dwarf WISE J011154.36-505343.2, index 12...
Working on dwarf WISEPA J012333.21+414203.9, index 13...
Working on dwarf CFBDS J013302.27+023128.4, index 14...
Working on dwarf WISE J014656.66+423410.0, index 15...
Working on dwarf WISEP J015010.86+382724.3, index 16...
Working on dwarf 2MASS J01550354+0950003, index 17...
Working on dwarf 

In [19]:
# reformat all_events
event_objects, event_sep, event_deltam = list(), list(), list()
for index, row in all_events.iterrows():
    # grab item 
    event_objects.append(row.object_name.item())
    
    # sep and delta_m
    if type(row.sep) is np.ndarray:
        event_sep.append(row.sep[0])
        event_deltam.append(row.delta_m[0])
    else:
        event_sep.append(row.sep)
        event_deltam.append(row.delta_m) 
        
all_events['object_name'] = event_objects
all_events['sep'] = event_sep
all_events['delta_m'] = event_deltam

In [90]:
all_events.sort_values(by=['delta_m'], ascending=True)

,object_name,sep,delta_m,bd_ra,bd_dec,ls_id,bs_ra,bs_dec,mag,time_of_min,gaia_id
0,L 726-8,0.061806,4.297247,24.77106,-17.948393,9906614735605912,24.771059,-17.948376,14.964031,2015.514031,NaN
0,WISEA J030601.66-033059.0,0.004497,4.701461,46.507697,-3.519314,9906620108443830,46.507697,-3.519313,19.899721,2018.733744,NaN
0,2MASSI J1807159+501531,0.042566,16.022464,271.816561,50.258207,9907737650204403,271.816572,50.258198,22.172455,2015.149897,NaN
0,WISE J041358.14-475039.3,0.068709,34.856879,63.493264,-47.842451,9906605406359798,63.493237,-47.842456,23.470789,2031.995893,NaN
0,2MASS J05441150-2433018,0.072282,38.340655,86.049254,-24.554795,9906612462491064,86.049248,-24.554715,19.171286,2021.953457,2.915188e+18
0,LEHPM 3396,0.198856,46.036767,53.589044,-49.887463,9906604845043726,53.589073,-49.887136,14.321546,2026.87885,4.832732e+18
2,L 726-8,0.680972,47.346667,24.780187,-17.947024,9906614735606300,24.780094,-17.946856,23.93515,2024.94319,NaN
0,WISE J222219.93+302601.4,0.052755,49.162770,335.583255,30.434634,9906632490814783,335.583084,30.434805,17.172844,2026.208077,1.894730e+18
0,SDSS J091659.09+512450.4,0.042377,51.171255,139.246517,51.414014,9907737926959557,139.246516,51.414026,23.350624,2015.092402,NaN
0,2MASSI J1743348+584411,0.073973,63.420977,265.895291,58.736249,9907739516014767,265.895329,58.736253,23.049599,2014.249144,NaN


Now let's take the objects with events and perform MCs on them (sample 500 times):

- There is one thing to note. I am checking that there is an event before the MC is run. This might initially seem silly considering I have already selected objects with a mass uncertainty < 1000 Mjup. A check might look redundant at first. But because I know the time of minimum separation, I am also changing the brown dwarf path to near that time. In doing so, I am recomputing the Horizons ephemerides using their module. The way the ephemerides are calculated, the vector output changes ever so slightly depending on the start time (which is different than the initial computation). This change of vector output slightly changes the eventual mass uncertaintly, and for objects with initially computed mass uncertainties near 1000 Mjup may be pushed over that limit. Therefore, I have to do this check beforehand, because I have a different start time (which will save time). And it's not like a mass uncertainty of 1000 Mjup is particularly helpful for these low mass objects anyways...

In [21]:
# make subset of dataframe with event object_names
event_rows = pd.DataFrame(all_obj.loc[all_obj['object_name'].isin(event_objects)])

mc_results = dict()
# iterate through all_events
for index, row in all_events.iterrows():
    print('Working on {}'.format(row.object_name))
    
    # find row of object in all_obj
    obj_row = pd.DataFrame(all_obj.loc[all_obj.object_name == row.object_name])
    
    # convert row to pd dataframe
    event_row = pd.DataFrame(row).T

    #print('obj row similarity', obj_row.object_name.item() == event_row.object_name.item())

    # brown dwarf observe date to create BrownDwarf instance
    t = time_to_good(obj_row.t0.item())

    # cater start and end of brown dwarf path to the event time (+- 0.1 years)
    t_min = round(float(event_row.time_of_min.item()), 2)

    t_start, t_end = time_to_good(t_min - 1, 'decimalyear'), time_to_good(t_min + 1, 'decimalyear')

    # instantiate BrownDwarf
    bd = BrownDwarf(obj_row, observ_date=t)
    bd_path = bd.find_path(start=t_start, end=t_end, step='6h')
    
    # find the background stars from the Legacy Survey
    stars = Fields(bd=bd)

    # find events with "new" object
    events = FindEvents(bd, stars, 1000)
    
    # perform the MC with 1000 samples and varying everything
    if len(events.event_table) > 0:
        mass_unc_list, sep_list, time_list, index_list = events.event_mcmc(samples=500, vary=['bd_ra', 'bd_dec', 'pi', 'mu_alpha', 'mu_delta'], prints=100)

        # add mass unc to dict
        mc_results[obj_row.object_name.item()] = mass_unc_list, sep_list, time_list, index_list

Working on WISE J004945.61+215120.0
Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on WISE J041358.14-475039.3


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on WISE J083337.83+005214.2
Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on WISE J085510.83-071442.5


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on WISE J085510.83-071442.5


...

/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on 2MASSI J1426316+155701
Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on SDSS J163355.23+010027.0
Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on 2MASSI J1743348+584411
Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on 2MASSI J1807159+501531
Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on LSR J1826+3014


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on WISE J215949.48-480854.9
Working on HIP 112422B
Working on sample #0...
Working on sample #100...
Working on sample #200...
Working on sample #300...
Working on sample #400...
Working on HIP 115819B
Working on SDSS J233358.42+005012.1


/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/judahluberto/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Convert sep_list in mc_results to mas (1000x currently have):

In [22]:
for i in mc_results:
    # make a list
    mc_results[i] = list(mc_results[i])
    
    # reassign
    mc_results[i][1] = np.array(mc_results[i][1]) * 1000

I also should convert the separations from arcseconds to milliarcseconds:

Now to save mc_results so I don't have to run this again in the future. I was thinking about good ways to do it, like np.savez etc. etc., but because I don't have too many objects to save, I'm going to save each object into their own individual file using savez:

In [58]:
for i in mc_results:
    print('Saving {}'.format(i))
    
    # get data
    deltam = mc_results[i][0]
    sep = mc_results[i][1]
    time = mc_results[i][2]
    index = mc_results[i][3]
    
    # file name and path
    filename = os.path.join('/Users/judahluberto/object_massuncertainty', i)
    
    np.savez(filename, deltam=deltam, sep=sep, time=time, index=index)

Saving WISE J004945.61+215120.0
Saving WISE J041358.14-475039.3
Saving WISE J083337.83+005214.2
Saving WISE J085510.83-071442.5
Saving WISE J220905.73+271143.9
Saving 2MASSW J0228110+253738
Saving 2MASSW J0832045-012835
Saving 2MASSW J1411175+393636
Saving SDSS J202820.32+005226.5
Saving WISE J222219.93+302601.4
Saving WISEA J001450.17-083823.4
Saving LHS 1070
Saving L 726-8
Saving WISEPA J020625.26+264023.6
Saving TVLM 831-161058
Saving PSO J043.5395+02.3995
Saving WISEA J030601.66-033059.0
Saving LEHPM 3396
Saving 2MASSI J0428510-225323
Saving 2MASS J05441150-2433018
Saving SDSS J091659.09+512450.4
Saving WISEPC J131106.24+012252.4
Saving 2MASSI J1426316+155701
Saving 2MASSI J1743348+584411
Saving 2MASSI J1807159+501531
Saving LSR J1826+3014
Saving HIP 112422B


Checking the first file (alphabetically) to see if saved correctly:

In [59]:
object_saved = np.load('/Users/judahluberto/object_massuncertainty/2MASSW J0228110+253738.npz')

object_saved['deltam']

array([136.48622555, 143.01641821, 151.04655332, 170.58780467,
       151.26242902, 144.95362151, 149.22767923, 131.21601071,
       151.5709125 , 171.67056153, 146.79820363, 133.29494339,
       157.28934853, 149.73701905, 147.9396125 , 149.12016615,
       162.30136174, 154.52552911, 159.20016408, 150.59770581,
       132.54405677, 149.33382826, 157.47815605, 152.79783904,
       152.62746748, 166.64278747, 157.71061725, 161.39384574,
       129.48487553, 141.55770114, 153.06655616, 150.49888237,
       151.98128762, 155.76589637, 164.11361164, 153.14538613,
       174.51246372, 156.00363552, 157.75393272, 168.23711236,
       156.99350473, 142.27542219, 154.72798902, 140.97595409,
       151.3510447 , 157.59319939, 154.94398703, 139.233917  ,
       139.8278428 , 179.14561934, 141.94849476, 147.97761782,
       157.66645647, 147.06890731, 145.21548894, 149.26460125,
       162.24773018, 150.11925359, 155.93777095, 165.14181104,
       156.48823578, 147.22783664, 150.19801631, 156.08

Import all the files and their data (so I don't have to redo the code that takes so long):

In [362]:
directory = '/Users/judahluberto/object_massuncertainty'

#mc_results = dict()
for file in os.listdir(directory):
    # ignore the DS_Store file
    if file.startswith('.DS_Store'):
        continue
        
    if file[:-4] not in list(all_obj.object_name):
        continue
    
    # do real work
    filename = os.path.join(directory, file)
    data = np.load(filename)
    
    # add to mc_results and change sep to mas
    #mc_results[file[:-4]] = data['deltam'], np.array(data['sep']) * 1000

#### I need to put this data of the objects into an ApJ accepted file format:

Create a pandas dataframe of the table I want to format:

In [36]:
# prepare all_the_bds to be put into a table -- organize by ra, dec later because mc_results depends on this order
obj_table = all_obj.copy()

obj_table = obj_table.drop(['t0', 'pi', 'mu_alpha', 'mu_delta', 'pm_ra', 'pm_dec', 'pm_mu_alpha', 'pm_mu_delta', 'pm_pi'], axis=1)

Add necessary data to this table:

- It's kind of ugly code because have to append to lots of lists, and then add those lists to the dataframe. I could make it cleaner and more beautiful...

In [134]:
# create duplicate array of all_events to only grab the lowest Mjup event
all_events_by_mjup = all_events.sort_values(by=['delta_m'], ascending=True)

# lists that will append
min_times, mags, seps, seps_e, delt_ms, delt_ms_e, times, times_e, gaia_sources = list(), list(), list(), list(), list(), list(), list(), list(), list()
for i in obj_table['object_name']:
    # if one of the objects with the events, add data
    if i in mc_results.keys() and i != 'SDSS J163355.23+010027.0':        
        # find index of where at in all_events
        index = list(all_events_by_mjup.object_name).index(i)
        
        # grab data from all_events
        #min_time = round(list(all_events_by_mjup['time_of_min'])[index], 2)
        mag = round(list(all_events_by_mjup['mag'])[index], 2)
        
        # calc data from MC
        delt_m, delt_m_e = np.mean(mc_results[i][0]), np.std(mc_results[i][0])
        sep, sep_e = np.mean(mc_results[i][1]), np.std(mc_results[i][1])
        time, time_e = np.mean(mc_results[i][2]), np.std(mc_results[i][2])

        # round calculated data
        sep, sep_e = round(sep, 2), round(sep_e, 2)
        delt_m, delt_m_e = round(delt_m, 2), round(delt_m_e, 2)
        time, time_e = round(time, 4), round(time_e, 4)
        
        # gaia id
        gaia_id = list(all_events_by_mjup['gaia_id'])[index]
        
        if ~np.isnan(gaia_id):
            gaia_source = True
            
        else:
            gaia_source = False
        
    # if not an object with an event, add "-"
    else:
        min_time, mag, sep, sep_e, delt_m, delt_m_e, time, time_e, gaia_source = '-', '-', '-', '-', '-', '-', '-', '-', '-'
        
    min_times.append(min_time)
    mags.append(mag)
    seps.append(sep)
    seps_e.append(sep_e)
    delt_ms.append(delt_m)
    delt_ms_e.append(delt_m_e)
    times.append(time)
    times_e.append(time_e)
    gaia_sources.append(gaia_source)
    
# add to df
#obj_table['t_min'] = min_times
obj_table['Mag_bs'] = mags

obj_table['Sep_min'] = seps
obj_table['Sep_min_e'] = seps_e
obj_table['Delta_m'] = delt_ms
obj_table['Delta_m_e'] = delt_ms_e

obj_table['t_min'] = times
obj_table['t_min_e'] = times_e

obj_table['Gaia_Source'] = gaia_sources

# now rearrange by ra, dec
obj_table = obj_table.sort_values('ra')

Add J-band (MKO system) photometry for the brown dwarfs):

In [136]:
k21_jmko_file = 'apjsabd107t17_mrt.txt'
b20_jmko_file = 'ajab84f4t1_mrt.txt'

ucs_jmko = ucs_obj_all[['J_MKO', 'Jerr_MKO', 'object_name']]

# open files
k21_jmko = pd.DataFrame(np.array(ascii.read(k21_jmko_file)))
b20_jmko = pd.DataFrame(np.array(ascii.read(b20_jmko_file)))

# get only column want
k21_jmko = k21_jmko[['JMKO', 'JMKe', 'FName']]
b20_jmko = b20_jmko[['Object', 'Jmag', 'e_Jmag']]

# rename columns
k21_jmko = k21_jmko.rename(columns = {'JMKe': 'JMKO_e', 'FName': 'object_name'})
b20_jmko = b20_jmko.rename(columns = {'Jmag': 'JMKO', 'e_Jmag': 'JMKO_e', 'Object': 'object_name'})
ucs_jmko = ucs_jmko.rename(columns = {'J_MKO': 'JMKO', 'Jerr_MKO': 'JMKO_e'})

# create list of all the magnitudes
jmko, jmko_e = list(), list()
for index, row in obj_table.iterrows():
    # find where from
    ref = row.Ref
    obj = row.object_name

    # locate row in J_MKO tables
    if ref == 'K21':
        obj_row = pd.DataFrame(k21_jmko.loc[k21_jmko.object_name == obj])

    if ref == 'B20':
        obj_row = pd.DataFrame(b20_jmko.loc[b20_jmko.object_name == obj])
        
    if ref == 'UCS':
        obj_row = pd.DataFrame(ucs_jmko.loc[ucs_jmko.object_name == obj])
        
    # some objects with astrometry in don't have a magnitude, append np.nan
    if len(obj_row) == 0:
        jmko.append(np.nan)
        jmko_e.append(np.nan)

        continue
             
    jmko.append(round(obj_row.JMKO.item(), 2))
    jmko_e.append(round(obj_row.JMKO_e.item(), 2))
            
# add columns to obj_table
obj_table['JMKO'] = jmko
obj_table['JMKO_e'] = jmko_e

# replace NaN with '-'
obj_table = obj_table.fillna('-')

In [137]:
# reorder columns
cols = ['object_name', 'Ref', 'ra', 'dec', 'JMKO', 'JMKO_e', 'Mag_bs', 't_min', 't_min_e', 'Sep_min', 'Sep_min_e', 'Delta_m', 'Delta_m_e', 'Gaia_Source']

obj_table = obj_table[cols]

# rename Mag
#obj_table = obj_table.rename(columns = {'Mag_BS': 'Mag_bs'})

In [138]:
obj_table[obj_table.t_min_e != '-']

,object_name,Ref,ra,dec,JMKO,JMKO_e,Mag_bs,t_min,t_min_e,Sep_min,Sep_min_e,Delta_m,Delta_m_e,Gaia_Source
683,WISEA J001450.17-083823.4,UCS,3.713295,-8.640502,14.43,0.03,21.55,2018.7386,0.0009,114.89,0.63,146.73,0.81,False
706,BRI 0021-0214,UCS,6.102194,-1.971402,11.75,0.01,20.7,2018.3536,0.0011,410.37,0.86,131.99,0.28,True
707,LHS 1070,UCS,6.182897,-27.133402,9.14,0.03,23.87,2013.0007,0.0,4687.73,2.59,865.55,0.48,False
9,WISE J004945.61+215120.0,K21,12.439534,21.855377,16.36,0.02,19.99,2023.7871,0.0014,3010.16,7.88,551.44,1.44,True
809,L 726-8,UCS,24.776296,-17.947600,-,-,14.96,2024.9474,0.0006,680.76,1.67,47.33,0.12,False
838,WISEPA J020625.26+264023.6,UCS,31.607691,26.672701,16.42,0.11,23.16,2022.7868,0.0025,1292.26,4.29,637.96,2.12,False
359,2MASSW J0228110+253738,B20,37.047154,25.627145,13.76,0.02,23.35,2018.4783,0.1091,161.64,10.44,150.63,9.73,True
894,TVLM 831-154910,UCS,42.549094,-1.858498,12.86,0.03,19.66,2019.9145,0.0028,332.54,0.24,429.37,0.31,True
902,PSO J043.5395+02.3995,UCS,43.554693,2.400902,15.92,0.01,13.75,2025.3894,0.0005,3034.24,5.87,534.17,1.03,True
911,WISEA J030601.66-033059.0,UCS,46.507894,-3.520098,14.4,0.03,19.9,2018.7305,0.0008,1.32,0.86,1.38,0.9,False


We are finally at the point where we can add to the file! Go us.

In [159]:
# format atb_with_results into ascii format wanted
top = 'Title: A Search for Predicted Astrometric Microlensing Events by Nearby Brown Dwarfs \n Authors: Luberto J., Martin E., McGill P., Leauthaud A., Skemer A., Lu J. \n Table: Catalog of Used Brown Dwarfs'
double_lines = '================================================================================'
byte_by_byte = 'Byte-by-byte Description of file: browndwarfs.txt'
single_lines = '--------------------------------------------------------------------------------'
headers = '   Bytes Format Units Label  Explanations'
#byte_descriptions = ' 1- 28 A28   ---     ObN      Name of Object \n 30- 42 A13.  ---     Ref.      Reference Source of Brown Dwarf \n 44- 50 F6.2   yrs     t_min      Date of Minimum Separation (- if no event) \n 52- 56 F4.2   ---     Mag.      Magnitude of Background Star (- if no event) \n 57- 61 F4.2   mas     Sep.      Minimum Separation of Brown Dwarf and Background Star \n 63- 68 F5.2   Mjup     Delta_M      Predicted Mass Uncertainty'

byte_descriptions =   """  1- 28 A28    ---     ObN           Name of Object 
 30- 42 A13.   ---     Ref           Reference Source of Object
 44- 48 F4.2   mag     JMKO          Object magnitude in MKO J-band
 50- 52 F3.2   mag     JMKO_e        Error in Object magnitude
 54- 58 F4.2   mag     Mag_bs        Magnitude of Background Star (- if no event) 
 60- 66 F6.2   yrs     t_min         Date of Minimum Separation (- if no event) 
 69- 73 F4.2   mas     Sep           Minimum Separation of Object and Background Star 
 75- 79 F4.2   mas     Sep_e         Error in Minimum Separation
 81- 86 F5.2   Mjup    Delta_M       Predicted Mass Uncertainty
 88- 92 F4.2   Mjup    Delta_M_e     Error in Predicted Mass Uncertainty
 94- 97 I1     ---     Gaia_Source   If Background Source is in Gaia"""

beginning = top + ' \n' + double_lines + ' \n' + byte_by_byte + ' \n' + single_lines + ' \n' + headers + ' \n' + single_lines + ' \n' + byte_descriptions + ' \n' + single_lines + ' \n'

with open('browndwarfs.txt', 'a') as file:
    file.write(beginning)
    
    for index, row in obj_table.iterrows():
        # grab itemsb
        object_name = row['object_name']
        ref = row['Ref']
        t_min = row['t_min']
        t_min_e = row['t_min_e']
        mag = row['Mag_bs']
        sep_min = row['Sep_min']
        sep_min_e = row['Sep_min_e']
        delta_m = row['Delta_m']
        delta_m_e = row['Delta_m_e']
        jmko = row['JMKO']
        jmko_e = row['JMKO_e']
        gaia_source = row['Gaia_Source']

        # format as needed
        object_name = "{:<28}".format(object_name)
        ref = "{:>13}".format(ref)
        t_min = "{:>7}".format(str(t_min))
        mag = "{:>5}".format(str(mag))
        sep_min = "{:>5}".format(str(sep_min))
        sep_min_e = "{:>5}".format(str(sep_min_e))
        delta_m = "{:>6}".format(str(delta_m))
        delta_m_e = "{:>5}".format(str(delta_m_e))
        jmko = "{:>5}".format(str(jmko))
        jmko_e = "{:>4}".format(str(jmko_e))
        
        if gaia_source != '-':
            gaia_source = int(gaia_source)
            
        gaia_source = "{:>3}".format(str(gaia_source))

        item = object_name + ' ' + ref + ' ' + jmko + ' ' + jmko_e + ' ' + mag + ' ' + t_min + ' ' + sep_min + ' ' + sep_min_e + ' ' + delta_m + ' ' + delta_m_e + ' ' + gaia_source + " \n"

        file.write(item)

I also want to output this in a form I can put in my LaTeX table...

In [139]:
# format atb_with_results into latex table
obj_table_events = obj_table[obj_table.t_min != '-']

in_still = {i: i in list(all_obj.object_name) for i in list(all_events.object_name)}

for index, row in obj_table_events.iterrows():
    # reformat row to combine the values and errors
    row['Sep_min'] = str(row['Sep_min']) + '$\pm$' + str(row['Sep_min_e'])
    row['Delta_m'] = str(row['Delta_m']) + '$\pm$' + str(row['Delta_m_e'])
    row['t_min'] = str(row['t_min']) + '$\pm$' + str(row['t_min_e'])
    
    # JMKO needs to be specialized a little
    if row['JMKO'] == '-':
        row['JMKO'] = '-'
        
    else:
        row['JMKO'] = str(row['JMKO']) + '$\pm$' + str(row['JMKO_e'])

    row = row.drop(['Sep_min_e', 'Delta_m_e', 'ra', 'dec', 't_min_e', 'JMKO_e'])

    # create line
    line = ''
    for value in row:
        line += str(value) + ' & '

    line = line[:-3] + r' \\'

    print(line)

WISEA J001450.17-083823.4 & UCS & 14.43$\pm$0.03 & 21.55 & 2018.7386$\pm$0.0009 & 114.89$\pm$0.63 & 146.73$\pm$0.81 & False \\
BRI 0021-0214 & UCS & 11.75$\pm$0.01 & 20.7 & 2018.3536$\pm$0.0011 & 410.37$\pm$0.86 & 131.99$\pm$0.28 & True \\
LHS 1070 & UCS & 9.14$\pm$0.03 & 23.87 & 2013.0007$\pm$0.0 & 4687.73$\pm$2.59 & 865.55$\pm$0.48 & False \\
WISE J004945.61+215120.0 & K21 & 16.36$\pm$0.02 & 19.99 & 2023.7871$\pm$0.0014 & 3010.16$\pm$7.88 & 551.44$\pm$1.44 & True \\
L 726-8 & UCS & - & 14.96 & 2024.9474$\pm$0.0006 & 680.76$\pm$1.67 & 47.33$\pm$0.12 & False \\
WISEPA J020625.26+264023.6 & UCS & 16.42$\pm$0.11 & 23.16 & 2022.7868$\pm$0.0025 & 1292.26$\pm$4.29 & 637.96$\pm$2.12 & False \\
2MASSW J0228110+253738 & B20 & 13.76$\pm$0.02 & 23.35 & 2018.4783$\pm$0.1091 & 161.64$\pm$10.44 & 150.63$\pm$9.73 & True \\
TVLM 831-154910 & UCS & 12.86$\pm$0.03 & 19.66 & 2019.9145$\pm$0.0028 & 332.54$\pm$0.24 & 429.37$\pm$0.31 & True \\
PSO J043.5395+02.3995 & UCS & 15.92$\pm$0.01 & 13.75 & 2025.389